In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm==0.8.5.post1

In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm==0.8.5.post1
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

In [31]:
import torch
torch.cuda.empty_cache()

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B-Base",
    max_seq_length = max_seq_length,
    load_in_4bit = False, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.7, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 07-10 14:04:08 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 07-10 14:04:09 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.7.1: Fast Qwen3 patching. Transformers: 4.53.1. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/Qwen3-4B-Base with actual GPU utilization = 69.34%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.7.1 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


# Load Dataset

In [7]:
import numpy as np
import pandas as pd
import json

import torch
import torch.nn.functional as F
from unsloth import FastLanguageModel
from datasets import Dataset, load_dataset

import os
from tqdm.auto import tqdm
from IPython.display import clear_output

In [8]:
dataset = load_dataset("deazzahra/8k_extracted_filings")["train"]

In [9]:
dataset

Dataset({
    features: ['input', 'summary_en'],
    num_rows: 560
})

In [25]:
system_message = """You are a helpful assistant specializing in summarizing SEC 8-K filings.
**Goal:** The summary should provide a quick understanding of the most important new developments at the company, suitable for an investor or stakeholder who needs to grasp the core information rapidly.
**Instructions:**
1. **Identify the Primary Event(s):** Determine the most significant event(s) disclosed in the 8-K filing. Prioritize items that indicate a major change in the company's financial condition, operations, control, or significant agreements. Common and highly relevant items include:
* Item 1.01: Entry into a Material Definitive Agreement
* Item 1.02: Termination of a Material Definitive Agreement
* Item 2.01: Completion of Acquisition or Disposition of Assets
* Item 2.02: Results of Operations and Financial Condition (e.g., earnings release)
* Item 2.05: Costs Associated with Exit or Disposal Activities (e.g., restructuring, layoffs)
* Item 4.01: Changes in Registrant's Certifying Accountant
* Item 4.02: Non-Reliance on Previously Issued Financial Statements
* Item 5.02: Departure of Directors or Certain Officers; Election of Directors; Appointment of Certain Officers; Compensatory Arrangements of Certain Officers
* Item 8.01: Other Events (if it describes a significant, non-routine occurrence)

2. **Extract Key Information:** For the identified primary event(s), extract the following essential details:
* **What happened?** (e.g., "Company X acquired Company Y," "Company Z entered a new credit agreement," "CEO resigned")
* **Who are the key parties involved?** (e.g., names of companies, individuals, financial institutions)
* **When did it happen?** (e.g., effective date, announcement date)
* **Why did it happen?** (e.g., strategic reasons, financial distress, operational changes, regulatory requirements)
* **What are the key terms or financial implications?** (e.g., acquisition price, value of agreement, expected costs, new debt amount, change in revenue, leadership changes)

###Instructions for answering:
* No wordiness, 350 words limit
* No further explanation, just summary
* The summary should be **abstractive** (rephrased and synthesized), not merely extractive (copy-pasting sentences).
* Keep the summary concise, ideally **between 3 to 7 sentences**.
* Use clear, professional, and easy-to-understand language. Avoid jargon where possible, or briefly explain it.
* Focus only on the *material* information. Do not include boilerplate legal text, administrative details, or general disclaimers unless they are central to the event.
* Do not include information that is explicitly marked as an "Exhibit" unless it is directly referenced and crucial for understanding the main event described in the body of the 8-K item.
* Do not hallucinate or invent any information. Only summarize what is explicitly stated in the provided 8-K text.


"""

user_template = """Below is the 8-K text. Please summarize it according to the instructions:

{}
"""

assistant_template = """

{}
"""

def apply_template_message(data):
    messages = [{
        "role": "system",
        "content": system_message
    }, {
        "role": "user",
        "content": user_template.format(data["input"])
    }, {
        "role": "assistant",
        "content": assistant_template.format(data["summary_en"])
    }]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False, enable_thinking=False)
    data["text"] = text
    return data

def apply_template_message_eval(data):
    messages = [{
        "role": "system",
        "content": system_message
    }, {
        "role": "user",
        "content": user_template.format(data["input"])
    }]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, e=False)
    data["text"] = text
    return data

In [11]:
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)

train_data = split_dataset["train"]
test_data = split_dataset["test"]

In [12]:
tokenizer.chat_template = """{% for message in messages %}
<|im_start|>{{ message['role'] }}
{{ message['content'] }}<|im_end|>
{% endfor %}
{% if add_generation_prompt %}
<|im_start|>assistant
{% endif %}
"""

In [13]:
train_data = train_data.map(apply_template_message)
test_data = test_data.map(apply_template_message_eval)

Map:   0%|          | 0/448 [00:00<?, ? examples/s]

Map:   0%|          | 0/112 [00:00<?, ? examples/s]

In [14]:
print(test_data[0]["text"])

<|im_start|>system
You are a helpful assistant specializing in summarizing SEC 8-K filings.
**Goal:** The summary should provide a quick understanding of the most important new developments at the company, suitable for an investor or stakeholder who needs to grasp the core information rapidly.
**Instructions:**
1. **Identify the Primary Event(s):** Determine the most significant event(s) disclosed in the 8-K filing. Prioritize items that indicate a major change in the company's financial condition, operations, control, or significant agreements. Common and highly relevant items include:
* Item 1.01: Entry into a Material Definitive Agreement
* Item 1.02: Termination of a Material Definitive Agreement
* Item 2.01: Completion of Acquisition or Disposition of Assets
* Item 2.02: Results of Operations and Financial Condition (e.g., earnings release)
* Item 2.05: Costs Associated with Exit or Disposal Activities (e.g., restructuring, layoffs)
* Item 4.01: Changes in Registrant's Certifying 

In [15]:
train_data

Dataset({
    features: ['input', 'summary_en', 'text'],
    num_rows: 448
})

In [16]:
test_data

Dataset({
    features: ['input', 'summary_en', 'text'],
    num_rows: 112
})

In [17]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_data,
    dataset_text_field = "text",
    max_seq_length = 8192,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 16,
        warmup_steps = 5,
        num_train_epochs = 1,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 5,

        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/448 [00:00<?, ? examples/s]

In [18]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 448 | Num Epochs = 3 | Total steps = 60
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 16 x 1) = 16
 "-____-"     Trainable parameters = 66,060,288 of 4,088,528,384 (1.62% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
5,1.296400
10,0.629900
15,0.360800
20,0.333400
25,0.329900
30,0.333100


Step,Training Loss
5,1.296400
10,0.629900
15,0.360800
20,0.333400
25,0.329900
30,0.333100
35,0.301700
40,0.299600
45,0.292800
50,0.305800


In [19]:
model.save_pretrained("qwen_finetuned_model")
tokenizer.save_pretrained("qwen_finetuned_model")

('qwen_finetuned_model/tokenizer_config.json',
 'qwen_finetuned_model/special_tokens_map.json',
 'qwen_finetuned_model/chat_template.jinja',
 'qwen_finetuned_model/vocab.json',
 'qwen_finetuned_model/merges.txt',
 'qwen_finetuned_model/added_tokens.json',
 'qwen_finetuned_model/tokenizer.json')

# Testing

In [20]:
from unsloth import FastLanguageModel
import torch

model_directory = "/content/qwen_finetuned_model"

max_seq_length = 8192
dtype = None
load_in_4bit = False

print(f"Memuat model dari folder: {model_directory}...")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_directory,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

print("\nModel berhasil dimuat!")

Memuat model dari folder: /content/qwen_finetuned_model...
==((====))==  Unsloth 2025.7.1: Fast Qwen3 patching. Transformers: 4.53.1. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.32G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/166 [00:00<?, ?B/s]


Model berhasil dimuat!


In [32]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tqdm import tqdm
import torch

def summarize_map_reduce(document_text, model, tokenizer, system_prompt, user_template):
    """
    Meringkas teks panjang menggunakan metode Map-Reduce dengan perbaikan CUDA.
    """
    print("--- Memulai Proses Map-Reduce ---")

    # 1. SPLIT: Bagi dokumen menjadi beberapa bagian kecil
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=4000,       # Safe for most models with 2048 token context
        chunk_overlap=200
    )
    chunks = text_splitter.split_text(document_text)
    print(f"Dokumen dipecah menjadi {len(chunks)} bagian.")

    # 2. MAP: Ringkasan tiap bagian
    chunk_summaries = []
    print("Tahap MAP: Meringkas setiap bagian...")

    for i, chunk in enumerate(tqdm(chunks, desc="Mapping Chunks")):
        try:
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_template.format(chunk)},
            ]
            inputs = tokenizer.apply_chat_template(
                messages,
                tokenize=True,
                add_generation_prompt=True,
                return_tensors="pt"
            ).to("cuda")

            attention_mask = (inputs != tokenizer.pad_token_id).long()

            with torch.no_grad():
                outputs = model.generate(
                    input_ids=inputs,
                    attention_mask=attention_mask,
                    max_new_tokens=256,  # Safer max length
                    use_cache=True,
                )

            summary = tokenizer.batch_decode(
                outputs[:, inputs.shape[1]:],
                skip_special_tokens=True
            )[0]

            chunk_summaries.append(summary)
            torch.cuda.empty_cache()

        except Exception as e:
            print(f"[!] Gagal merangkum chunk {i+1}: {e}")
            continue

    # 3. REDUCE: Gabungkan semua ringkasan menjadi satu ringkasan akhir
    print("\nTahap REDUCE: Membuat ringkasan akhir...")
    combined_summaries = "\n".join(chunk_summaries)

    reduce_prompt = f"""
You are a helpful assistant specializing in summarizing financial disclosures such as SEC 8-K filings.

Below are partial summaries generated from different sections of a long financial report. Your task is to synthesize these into a single, coherent, and concise final summary that captures the most important material developments.

**Instructions:**
- Focus on *what happened*, *who was involved*, *when it occurred*, and any *financial or operational implications*.
- Avoid redundancy, keep the summary **abstractive**, and use clear and professional language.
- Final summary should be between **3 to 7 sentences** and not exceed **350 words**.
- Do not include irrelevant details or disclaimers unless they are central to the event.

**Partial Summaries:**
{combined_summaries}
    """

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": reduce_prompt},
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")

    attention_mask = (inputs != tokenizer.pad_token_id).long()

    with torch.no_grad():
        final_outputs = model.generate(
            input_ids=inputs,
            attention_mask=attention_mask,
            max_new_tokens=256,
            use_cache=True,
        )

    final_summary = tokenizer.batch_decode(
        final_outputs[:, inputs.shape[1]:],
        skip_special_tokens=True
    )[0]

    torch.cuda.empty_cache()
    print("--- Proses Selesai ---")
    return final_summary


In [ ]:
import random

# Ambil satu sampel acak dari data tes
sample_index = random.choice(range(len(test_data)))
sample_data = test_data[sample_index]

input_text = sample_data["input"]
reference_summary = sample_data["summary_en"]

# Panggil fungsi Map-Reduce untuk mendapatkan ringkasan
# Ini akan mengatasi masalah input yang terpotong
generated_summary = summarize_map_reduce(
    document_text=input_text,
    model=model,
    tokenizer=tokenizer,
    system_prompt=system_message,
    user_template=user_template
)

# --- Tampilkan Hasil Perbandingan ---
print(f"\n===== CONTOH {sample_index} =====")
print("\n📜 TEKS INPUT (potongan):")
print(input_text[:500] + "...")
print("\n✅ RINGKASAN REFERENSI (dari dataset):")
print(reference_summary)
print("\n✨ RINGKASAN HASIL MODEL (DENGAN MAP-REDUCE):")
print(generated_summary)

In [ ]:
import torch
import random
from IPython.display import display, Markdown

# Set model to evaluation mode
model.eval()

# Run inference on 3 random samples
for i in random.sample(range(len(test_data)), 3):
    input_text = test_data[i]["input"]
    reference_summary = test_data[i]["summary_en"]

    # Build message with system and user prompt
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_template.format(input_text)}
    ]

    # Tokenize input
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")

    # Generate output
    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=350,
        use_cache=True,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode model output
    generated_summary = tokenizer.batch_decode(
        outputs[:, inputs.shape[1]:], skip_special_tokens=True)[0]

    # Print Results
    print(f"=======EXAMPLE {i+1}========")
    print("Input text")
    print(input_text[:500]+"...")
    print("\n Reference:")
    print(reference_summary)
    print("\n from model:")
    print(generated_summary)